<a href="https://colab.research.google.com/github/szucshey/osm-data-exploration/blob/main/climate/climate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install osm2geojson geopandas regionmask netCDF4 basemap --quiet

In [31]:
import folium
import requests
import json
import osm2geojson
import geopandas as gpd
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
import regionmask
import numpy as np
import PIL
from datetime import datetime, timedelta, date
from mpl_toolkits.basemap import Basemap

In [33]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
    [out:json];
    rel["ISO3166-1"~"^HU"]
    [admin_level=2]
    [type=boundary]
    [boundary=administrative];
    out geom;
    """

response = requests.get(overpass_url, params={'data': overpass_query})

#Az API válasz lementése egy json fileba
output = open("geojson.json", "w")
json.dump(response.json(), output)
output.close()

In [34]:
data = json.load(open("geojson.json", "r"))
geojson = osm2geojson.json2geojson(data)
gdf = gpd.GeoDataFrame.from_features(geojson)

In [36]:
ds = xr.open_dataarray("drive/MyDrive/climate/rr_ens_mean_0.1deg_reg_1950-1964_v25.0e.nc")
ds = ds.sel(time=ds['time'], longitude=slice(15.96, 23.12), latitude=slice(45.55, 48.73))
ds.to_netcdf("sliced_dataset.nc")

In [38]:
ds_test = Dataset("sliced_dataset.nc")

lats = ds_test.variables['latitude'][:]
lons = ds_test.variables['longitude'][:]
dates = ds_test.variables['time'][:]
rain = ds_test.variables['rr'][:]

lon, lat = np.meshgrid(lons, lats)

In [41]:
#longitude=slice(15.96, 23.12), latitude=slice(45.55, 48.73))
start_date = date(1950, 1, 1)

for i in range(1852, len(dates)):
  end_date = start_date + timedelta(days=i)
  y, m, d = end_date.strftime("%Y"), end_date.strftime("%m"), end_date.strftime("%d")
  end_date_str = y + "-" + m + "-" + d

  map = Basemap(llcrnrlon=16.06, llcrnrlat=45.65, urcrnrlon=23.02, urcrnrlat=48.63, projection='merc', resolution='i')

  x,y = map(lon, lat)
  map.drawcountries()

  plt.title("Csapadék mennyiség: {0}".format(end_date_str))

  color_scheme = map.pcolormesh(x,y, np.squeeze(rain[i, :, :]), cmap='cubehelix_r', vmin=0, vmax=100)
  color_bar = map.colorbar(color_scheme, location='right', pad='5%')
  color_bar.set_label('Csapadék mennyisége (mm)', labelpad=5)
  plt.savefig("drive/MyDrive/climate/test_output/{0}.jpg".format(i+1), dpi=80)
  plt.clf()


<Figure size 432x288 with 0 Axes>

In [44]:
frames = []

for i in range(len(dates)):
  frame = PIL.Image.open("drive/MyDrive/climate/test_output/{0}.jpg".format(str(i+1)))
  frames.append(frame)

frames[0].save('drive/MyDrive/climate/gifs/test.gif', 
               format='GIF',
               append_images = frames[1: ], 
               save_all = True,
               duration=10, loop=0)